# Part 2: Simple model

Running all models at once might crash the kernel.

# Task 0: Choice of label groups

political and reliable has been chosen as "real news", while the rest as "fake news". some categories like unknown or unreliable might include legitimate articles which were put in their categories simply due to the lack of validation and confirmation. This might make it difficult for the model to correctly categorize these articles.


In [1]:
import numpy as np
from scipy.sparse import csr_matrix, hstack, save_npz
data = np.load('../Simple_995.npz', allow_pickle=True)
data_BBC = np.load('../Simple_995_BBC.npz', allow_pickle=True)


# task 1: Simple model baselines

There are two baseline models, one with one-hot encoding, the other a Bag of Words encoding. both use logistic regression, as it is the simplest classification model.

### logistic regression with one-hot encoding:

This is the simplest model which made sence that we could think of. It uses one-hot representations of the articles' vocabularies to classify them. However, this representation does not take frequency into consideration, making it ineffective.

In [2]:
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle


#load features
X_train=data['X_train_content_ONEHOT'].item()
X_val=data['X_val_content_ONEHOT'].item()
Y_train = data['Y_train'].ravel()
Y_val = data['Y_val'].ravel()

#fit,predict and evaluate
model = LogisticRegression(tol=1e-3, max_iter=1000,penalty = None)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
print(classification_report(Y_val, Y_pred))

with open("../simple_model_ONEHOT.pkl",'wb') as f:
    pickle.dump(model,f)

del model, X_train, X_val, Y_train, Y_val, Y_pred

              precision    recall  f1-score   support

           0       0.87      0.22      0.35     43331
           1       0.54      0.96      0.69     41194

    accuracy                           0.58     84525
   macro avg       0.70      0.59      0.52     84525
weighted avg       0.71      0.58      0.52     84525



### Logistic regression with Bag of words encoding:

This is the simple model baseline takes frequency into consideration, and yields considerably better results.

This is the simple model used from here on out, as it is simple, but not too much so.

In [3]:
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle


#load features
X_train=data['X_train_content_BOW'].item()
X_val=data['X_val_content_BOW'].item()
Y_train = data['Y_train'].ravel()
Y_val = data['Y_val'].ravel()

#fit,predict and evaluate
model = LogisticRegression(tol=1e-3, max_iter=1000,penalty = None)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
print(classification_report(Y_val, Y_pred))

    
del model, X_train, X_val, Y_train, Y_val, Y_pred

              precision    recall  f1-score   support

           0       0.86      0.89      0.88     43331
           1       0.88      0.84      0.86     41194

    accuracy                           0.87     84525
   macro avg       0.87      0.87      0.87     84525
weighted avg       0.87      0.87      0.87     84525



# Task 2: Which column features might be useful

From the data exploration, it is clear that some domains produce much more real/fake news than others. This would make domain a good indicator of fake/real news.

Furthermore, the title of the article would also be helpful; "fake" articles, especially, especially ones like "hate", "extreme bias", "clickbait", etc. will have distinct titles that are provocative, misleading or exaggerated. This feature would therefore help distinguish fake/real news.

The rest of the columns have too many empty cells to be used as features.

### Domain:

Adding domain features to the simple model, we can see that the model gives a almost perfect prediction, which is surprising.

Training the simple model with only domian features, we get a perfect prediction, indicating that the domain of an article is directly tied to it being real or fake.

Because of this, it would not make sense to include domain in the model, at it effectively serves as an answer.

In [2]:
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#load features
X_train_content=data['X_train_content_BOW'].item()
X_val_content=data['X_val_content_BOW'].item()
X_train_domain = data['X_train_domain'].item()
X_val_domain = data['X_val_domain'].item()

Y_train = data['Y_train'].ravel()
Y_val = data['Y_val'].ravel()

#combine content and domain features
X_train = hstack([X_train_content,X_train_domain])
X_val = hstack([X_val_content, X_val_domain])

#train predict and evaluate with content and domain
model = LogisticRegression(tol=1e-3, max_iter=1000,penalty = None)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
print("Model using Content and Domain")
print(classification_report(Y_val, Y_pred))

#train predict and evaluate with only domain
model1 = LogisticRegression(tol=1e-3)
model1.fit(X_train_domain, Y_train)
Y_pred = model1.predict(X_val_domain)
print("Model using Domain only")
print(classification_report(Y_val, Y_pred))

del model,model1, X_train_content, X_val_content,X_train_domain, X_val_domain, Y_train, Y_val, Y_pred


Model using Content and Domain
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     43331
           1       0.99      0.98      0.98     41194

    accuracy                           0.99     84525
   macro avg       0.99      0.99      0.99     84525
weighted avg       0.99      0.99      0.99     84525

Model using Domain only
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     43331
           1       1.00      1.00      1.00     41194

    accuracy                           1.00     84525
   macro avg       1.00      1.00      1.00     84525
weighted avg       1.00      1.00      1.00     84525



### Title:

adding title features gives much better performance. This indicates, that the title of an article helps distinguish real news from fake ones. It therefore makes sence to include title.

In [2]:
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#load features
X_train_content=data['X_train_content_BOW'].item()
X_val_content=data['X_val_content_BOW'].item()
X_train_title = data['X_train_title'].item()
X_val_title = data['X_val_title'].item()
Y_train = data['Y_train'].ravel()
Y_val = data['Y_val'].ravel()

#combine content and domain features
X_train = hstack([X_train_content,X_train_title])
X_val = hstack([X_val_content, X_val_title])

#train predict and evaluate with content and domain
model = LogisticRegression(tol=1e-3, max_iter=1000,penalty = None)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
print("Model using Content and Title")
print(classification_report(Y_val, Y_pred))
del model, X_train_content, X_val_content,X_train_title, X_val_title, Y_train, Y_val, Y_pred


Model using Content and Title
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     43331
           1       0.90      0.89      0.89     41194

    accuracy                           0.90     84525
   macro avg       0.90      0.90      0.90     84525
weighted avg       0.90      0.90      0.90     84525



# task 3: Simple model with added BBC articles

Adding the BBC articles, there isnt much difference in the result. However, since the dataset had slightly more fake articles, the BBC articles are included. They make the dataset just a bit more balanced, and more data can't hurt.

In [2]:
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

#load features
X_train=data_BBC['X_train'].item()
X_val = data_BBC['X_val'].item()
Y_train = data_BBC['Y_train'].ravel()
Y_val = data_BBC['Y_val'].ravel()

#fit,predict and evaluate
model = LogisticRegression(tol=1e-3, max_iter=1000,penalty = None)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_val)
print(classification_report(Y_val, Y_pred))

with open("../simple_model_BOW.pkl",'wb') as f:
    pickle.dump(model,f)
    
del model, X_train, X_val, Y_train, Y_val, Y_pred

              precision    recall  f1-score   support

           0       0.86      0.89      0.88     43081
           1       0.89      0.85      0.87     42014

    accuracy                           0.87     85095
   macro avg       0.87      0.87      0.87     85095
weighted avg       0.87      0.87      0.87     85095

